In [1]:
import pandas as pd
import numpy as np
#from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib.gridspec import GridSpec
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from warnings import filterwarnings
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, silhouette_samples
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram
#from factor_analyzer import FactorAnalyzer
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
#import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from itertools import combinations
from kmodes.kprototypes import KPrototypes
from collections import Counter
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE
### so that u dont have warnings
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.cluster import DBSCAN
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

In [2]:
df_1 = pd.read_csv("Predict Hair Fall.csv")

In [3]:
X = df_1.drop(columns=['Hair Loss'])
Y = df_1['Hair Loss']

In [4]:
from category_encoders import TargetEncoder

In [5]:
from sklearn.preprocessing import OneHotEncoder

# Define your target encoder columns

target_encoder_cols = X.select_dtypes('O').columns.tolist()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Perform frequency encoding for each column
for col in target_encoder_cols:
    target_encoder = TargetEncoder()

    # Fit and transform the target encoder on the training data
    X_train[target_encoder_cols] = target_encoder.fit_transform(X_train[target_encoder_cols], y_train)

    # Transform the test data using the target encoder fitted on the training data
    X_test[target_encoder_cols] = target_encoder.transform(X_test[target_encoder_cols])


In [6]:
from sklearn.cluster import KMeans

random_seed = 42
n_clusters = 2  # Assuming you want to cluster into 2 groups
kmeans = KMeans(n_clusters=n_clusters, random_state=random_seed)
kmeans.fit(X_train)
predictions = kmeans.labels_

X_train['cluster_label'] = predictions

X_train_1 = X_train[X_train['cluster_label'] == 0]
X_train_2 = X_train[X_train['cluster_label'] == 1]

indexes_1 = X_train_1.index
indexes_2 = X_train_2.index

y_train_1 = y_train.loc[indexes_1]
y_train_2 = y_train.loc[indexes_2]


In [7]:
x_train_cluster_1 , x_test_cluster_1 , y_train_cluster_1 , y_test_cluster_1 = train_test_split(X_train_1,y_train_1,test_size=0.2,random_state=42)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Create Pipeline with feature selection and XGBoost for classification
pipeline_xgb = Pipeline([
    ('feature_selection', SelectFromModel(XGBClassifier())),
    ('xgb', XGBClassifier())
])

# Define parameter grid for hyperparameter tuning for XGBoost
param_grid_xgb = {
    'feature_selection__estimator__alpha': [0.1, 1.0, 10.0],
    'feature_selection__estimator__l1_ratio': [0.1, 0.5, 0.9],
    'xgb__learning_rate': [0.1, 0.01, 0.001],
    'xgb__n_estimators': [100, 500, 1000],
    'xgb__max_depth': [3, 5, 7]
}

# Perform GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)

# Fit GridSearchCV for XGBoost
grid_search_xgb.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best model for XGBoost
best_model_xgb_1 = grid_search_xgb.best_estimator_

# Get the selected features for XGBoost
selected_features_xgb = x_train_cluster_1.columns[best_model_xgb_1.named_steps['feature_selection'].get_support()]

# Evaluate the best model for XGBoost
y_train_pred_xgb = best_model_xgb_1.predict(x_train_cluster_1)
y_test_pred_xgb = best_model_xgb_1.predict(x_test_cluster_1)

# Accuracy
accuracy_train_xgb = accuracy_score(y_train_cluster_1, y_train_pred_xgb)
accuracy_test_xgb = accuracy_score(y_test_cluster_1, y_test_pred_xgb)

print("Selected Features (XGBoost):", selected_features_xgb)
print("Accuracy on Training set (XGBoost):", accuracy_train_xgb)
print("Accuracy on Testing set (XGBoost):", accuracy_test_xgb)
print(grid_search_xgb.best_params_)


Selected Features (XGBoost): Index(['Hormonal Changes', 'Medical Conditions', 'Medications & Treatments',
       'Nutritional Deficiencies ', 'Poor Hair Care Habits ', 'Smoking',
       'Weight Loss '],
      dtype='object')
Accuracy on Training set (XGBoost): 0.5709876543209876
Accuracy on Testing set (XGBoost): 0.5308641975308642
{'feature_selection__estimator__alpha': 0.1, 'feature_selection__estimator__l1_ratio': 0.1, 'xgb__learning_rate': 0.001, 'xgb__max_depth': 3, 'xgb__n_estimators': 100}


In [10]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

# Predictions for Cluster 1
y_train_pred_xgb_cluster_1 = best_model_xgb_1.predict(x_train_cluster_1)
y_test_pred_xgb_cluster_1 = best_model_xgb_1.predict(x_test_cluster_1)

# Accuracy for Cluster 1
accuracy_train_xgb_cluster_1 = accuracy_score(y_train_cluster_1, y_train_pred_xgb_cluster_1)
accuracy_test_xgb_cluster_1 = accuracy_score(y_test_cluster_1, y_test_pred_xgb_cluster_1)

# ROC AUC Score for Cluster 1
roc_auc_train_xgb_cluster_1 = roc_auc_score(y_train_cluster_1, best_model_xgb_1.predict_proba(x_train_cluster_1)[:, 1])
roc_auc_test_xgb_cluster_1 = roc_auc_score(y_test_cluster_1, best_model_xgb_1.predict_proba(x_test_cluster_1)[:, 1])

# Classification Report for Cluster 1
class_report_train_xgb_cluster_1 = classification_report(y_train_cluster_1, y_train_pred_xgb_cluster_1)
class_report_test_xgb_cluster_1 = classification_report(y_test_cluster_1, y_test_pred_xgb_cluster_1)

print("Selected Features (XGBoost) for Cluster 1:", selected_features_xgb)
print("Accuracy on Training set (XGBoost) for Cluster 1:", accuracy_train_xgb_cluster_1)
print("Accuracy on Testing set (XGBoost) for Cluster 1:", accuracy_test_xgb_cluster_1)
print("ROC AUC Score on Training set (XGBoost) for Cluster 1:", roc_auc_train_xgb_cluster_1)
print("ROC AUC Score on Testing set (XGBoost) for Cluster 1:", roc_auc_test_xgb_cluster_1)
print("Classification Report on Training set (XGBoost) for Cluster 1:\n", class_report_train_xgb_cluster_1)
print("Classification Report on Testing set (XGBoost) for Cluster 1:\n", class_report_test_xgb_cluster_1)
print(grid_search_xgb.best_params_)



Selected Features (XGBoost) for Cluster 1: Index(['Hormonal Changes', 'Medical Conditions', 'Medications & Treatments',
       'Nutritional Deficiencies ', 'Poor Hair Care Habits ', 'Smoking',
       'Weight Loss '],
      dtype='object')
Accuracy on Training set (XGBoost) for Cluster 1: 0.5709876543209876
Accuracy on Testing set (XGBoost) for Cluster 1: 0.5308641975308642
ROC AUC Score on Training set (XGBoost) for Cluster 1: 0.6587789228076998
ROC AUC Score on Testing set (XGBoost) for Cluster 1: 0.47582619339045273
Classification Report on Training set (XGBoost) for Cluster 1:
               precision    recall  f1-score   support

           0       0.57      1.00      0.73       185
           1       0.00      0.00      0.00       139

    accuracy                           0.57       324
   macro avg       0.29      0.50      0.36       324
weighted avg       0.33      0.57      0.42       324

Classification Report on Testing set (XGBoost) for Cluster 1:
               precisio

In [11]:
x_train_cluster_2 , x_test_cluster_2 , y_train_cluster_2 , y_test_cluster_2 = train_test_split(X_train_2,
                                                                                    y_train_2,test_size=0.2,random_state=42)

In [12]:
# Create Pipeline with feature selection and XGBoost for classification
pipeline_xgb = Pipeline([
    ('feature_selection', SelectFromModel(XGBClassifier())),
    ('xgb', XGBClassifier())
])

# Define parameter grid for hyperparameter tuning for XGBoost
param_grid_xgb = {
    'feature_selection__estimator__alpha': [0.1, 1.0, 10.0],
    'feature_selection__estimator__l1_ratio': [0.1, 0.5, 0.9],
    'xgb__learning_rate': [0.1, 0.01, 0.001],
    'xgb__n_estimators': [100, 500, 1000],
    'xgb__max_depth': [3, 5, 7]
}

# Perform GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(pipeline_xgb, param_grid_xgb, cv=5, scoring='accuracy', n_jobs=-1)

# Fit GridSearchCV for XGBoost
grid_search_xgb.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best model for XGBoost
best_model_xgb_2 = grid_search_xgb.best_estimator_

# Get the selected features for XGBoost
selected_features_xgb = x_train_cluster_2.columns[best_model_xgb_2.named_steps['feature_selection'].get_support()]

# Evaluate the best model for XGBoost
y_train_pred_xgb = best_model_xgb_2.predict(x_train_cluster_2)
y_test_pred_xgb = best_model_xgb_2.predict(x_test_cluster_2)

# Accuracy
accuracy_train_xgb = accuracy_score(y_train_cluster_2, y_train_pred_xgb)
accuracy_test_xgb = accuracy_score(y_test_cluster_2, y_test_pred_xgb)

print("Selected Features (XGBoost):", selected_features_xgb)
print("Accuracy on Training set (XGBoost):", accuracy_train_xgb)
print("Accuracy on Testing set (XGBoost):", accuracy_test_xgb)
print(grid_search_xgb.best_params_)


Selected Features (XGBoost): Index(['Id', 'Medical Conditions', 'Medications & Treatments',
       'Nutritional Deficiencies ', 'Age'],
      dtype='object')
Accuracy on Training set (XGBoost): 0.6476190476190476
Accuracy on Testing set (XGBoost): 0.5316455696202531
{'feature_selection__estimator__alpha': 10.0, 'feature_selection__estimator__l1_ratio': 0.1, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 3, 'xgb__n_estimators': 100}


In [18]:
# Predictions for Cluster 2
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

y_train_pred_xgb_cluster_2 = best_model_xgb_1.predict(x_train_cluster_2)
y_test_pred_xgb_cluster_2 = best_model_xgb_1.predict(x_test_cluster_2)

# Accuracy for Cluster 2
accuracy_train_xgb_cluster_2 = accuracy_score(y_train_cluster_2, y_train_pred_xgb_cluster_2)
accuracy_test_xgb_cluster_2 = accuracy_score(y_test_cluster_2, y_test_pred_xgb_cluster_2)

# ROC AUC Score for Cluster 2
roc_auc_train_xgb_cluster_2 = roc_auc_score(y_train_cluster_2, best_model_xgb_2.predict_proba(x_train_cluster_2)[:, 1])
roc_auc_test_xgb_cluster_2 = roc_auc_score(y_test_cluster_2, best_model_xgb_2.predict_proba(x_test_cluster_2)[:, 1])

# Classification Report for Cluster 2
class_report_train_xgb_cluster_2 = classification_report(y_train_cluster_2, y_train_pred_xgb_cluster_2)
class_report_test_xgb_cluster_2 = classification_report(y_test_cluster_2, y_test_pred_xgb_cluster_2)

print("Selected Features (XGBoost) for Cluster 2:", selected_features_xgb)
print("Accuracy on Training set (XGBoost) for Cluster 2:", accuracy_train_xgb_cluster_2)
print("Accuracy on Testing set (XGBoost) for Cluster 2:", accuracy_test_xgb_cluster_2)
print("ROC AUC Score on Training set (XGBoost) for Cluster 2:", roc_auc_train_xgb_cluster_2)
print("ROC AUC Score on Testing set (XGBoost) for Cluster 2:", roc_auc_test_xgb_cluster_2)
print("Classification Report on Training set (XGBoost) for Cluster 2:\n", class_report_train_xgb_cluster_2)
print("Classification Report on Testing set (XGBoost) for Cluster 2:\n", class_report_test_xgb_cluster_2)
print(grid_search_xgb.best_params_)


Selected Features (XGBoost) for Cluster 2: Index(['Id', 'Medical Conditions', 'Medications & Treatments',
       'Nutritional Deficiencies ', 'Age'],
      dtype='object')
Accuracy on Training set (XGBoost) for Cluster 2: 0.46984126984126984
Accuracy on Testing set (XGBoost) for Cluster 2: 0.46835443037974683
ROC AUC Score on Training set (XGBoost) for Cluster 2: 0.7677010843178508
ROC AUC Score on Testing set (XGBoost) for Cluster 2: 0.573037323037323
Classification Report on Training set (XGBoost) for Cluster 2:
               precision    recall  f1-score   support

           0       0.47      1.00      0.64       148
           1       0.00      0.00      0.00       167

    accuracy                           0.47       315
   macro avg       0.23      0.50      0.32       315
weighted avg       0.22      0.47      0.30       315

Classification Report on Testing set (XGBoost) for Cluster 2:
               precision    recall  f1-score   support

           0       0.47      1.00 

In [19]:
selected_features_xgb

Index(['Id', 'Medical Conditions', 'Medications & Treatments',
       'Nutritional Deficiencies ', 'Age'],
      dtype='object')